In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.font_manager import FontProperties
import pyscisci.all as pyscisci
from brokenaxes import brokenaxes
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
from scipy.stats import norm, ks_2samp
from collections import defaultdict
EF = FontProperties(family = 'Arial')

%matplotlib inline

In [42]:
Publication = r' '
Pdf = pd.read_csv(Publication)
print(Pdf .shape)
del Pdf['Unnamed: 0']
Pdf .columns

(993998, 5)


Index(['work_id', 'PublishedYear', 'DocumentType', 'Journal'], dtype='object')

In [48]:
Pdf = Pdf[Pdf['PublishedYear']<=2023]
# Article Type
Pdf = Pdf[Pdf['DocumentType']=='article']
print(Pdf .shape)

(407120, 4)


In [49]:
Works_authorship = r' '
WAdf = pd.read_csv(Works_authorship)
del WAdf['Unnamed: 0']
print(WAdf .shape)
WAdf .columns

(3419221, 4)


Index(['work_id', 'author_position', 'author_id', 'institution_id'], dtype='object')

In [ ]:
WAdf = WAdf .drop_duplicates(subset = ['work_id','author_id'])
print(WAdf .shape)

(2484365, 4)


In [ ]:
WAdf['institution_id'] = WAdf['institution_id'].fillna(0)
WAdf_na = WAdf[WAdf['institution_id']==0]
WAdf = WAdf[~WAdf['work_id'].isin(WAdf_na['work_id'].drop_duplicates())]
print(WAdf .shape)

(1653692, 4)


In [52]:
Pdf = Pdf[Pdf['work_id'].isin(WAdf['work_id'].drop_duplicates())]
print(Pdf .shape)

(407120, 4)


In [53]:
Pdf['Journal'].value_counts()

Journal
Nature                                             149786
Science                                            134035
Proceedings of the National Academy of Sciences    123299
Name: count, dtype: int64

In [56]:
Pdf['DocumentType'].value_counts()

DocumentType
article    407120
Name: count, dtype: int64

In [55]:
WAdf = WAdf[WAdf['work_id'].isin(Pdf['work_id'].drop_duplicates())]
print(WAdf .shape)

(1509324, 4)


In [57]:
WAdf_first_author = WAdf[WAdf['author_position']=='first']
print(WAdf_first_author .shape)

WAdf_first_author = WAdf_first_author .merge(Pdf[['work_id','Journal']],
                                             how = 'left', left_on = 'work_id', right_on = 'work_id')
Num_first_author = WAdf_first_author .groupby(['Journal'])['author_id'].nunique().reset_index()
print(Num_first_author)

(407120, 4)
                                           Journal  author_id
0                                           Nature      91387
1  Proceedings of the National Academy of Sciences      99099
2                                          Science      73190


In [58]:
WAdf_last_author = WAdf[WAdf['author_position']=='last']
print(WAdf_last_author .shape)

WAdf_last_author = WAdf_last_author .merge(Pdf[['work_id','Journal']],
                                             how = 'left', left_on = 'work_id', right_on = 'work_id')
Num_last_author = WAdf_last_author .groupby(['Journal'])['author_id'].nunique().reset_index()
print(Num_last_author)

(251822, 4)
                                           Journal  author_id
0                                           Nature      53921
1  Proceedings of the National Academy of Sciences      55438
2                                          Science      36793


In [59]:
WAdf = WAdf .merge(Pdf[['work_id','Journal']],
                   how = 'left', left_on = 'work_id', right_on = 'work_id')
Num_author = WAdf .groupby(['Journal'])['author_id'].nunique().reset_index()
print(Num_author)

                                           Journal  author_id
0                                           Nature     270873
1  Proceedings of the National Academy of Sciences     386580
2                                          Science     213418


In [61]:

WAdf_first_institution = WAdf[['work_id','author_position','institution_id','Journal']]
WAdf_first_institution = WAdf_first_institution[WAdf_first_institution['author_position']=='first']
print(WAdf_first_institution .shape)

Num_first_institution = WAdf_first_institution .groupby(['Journal'])['institution_id'].nunique().reset_index()
print(Num_first_institution)

(407120, 4)
                                           Journal  institution_id
0                                           Nature            9493
1  Proceedings of the National Academy of Sciences            6437
2                                          Science            7295


In [64]:
WAdf .columns
WAdf_institution = WAdf[['work_id','institution_id','Journal']]
print(WAdf_institution .shape)

Num_institution = WAdf_institution .groupby(['Journal'])['institution_id'].nunique().reset_index()
print(Num_institution)

(1509324, 3)
                                           Journal  institution_id
0                                           Nature           13367
1  Proceedings of the National Academy of Sciences           11842
2                                          Science           10768


In [67]:
print(Pdf .shape)
Num_year = Pdf.groupby(['Journal'])['PublishedYear'].nunique().reset_index()
print(Num_year)

(407120, 4)
                                           Journal  PublishedYear
0                                           Nature            155
1  Proceedings of the National Academy of Sciences            109
2                                          Science            144


In [69]:
WAdf_first_author .columns

Numpub_first_author_journal = WAdf_first_author .groupby(['Journal','author_id'])['work_id'].nunique().reset_index()
print(Numpub_first_author_journal .shape)

(263676, 3)


In [72]:
Numpub_journal_first_author_distribution = Numpub_first_author_journal.groupby(['Journal','work_id'])['author_id'].nunique().reset_index()
print(Numpub_journal_first_author_distribution .shape)

(172, 3)


In [73]:
Numpub_journal_first_author_distribution = Numpub_journal_first_author_distribution[Numpub_journal_first_author_distribution['work_id']<=20]
print(Numpub_journal_first_author_distribution .shape)

(60, 3)


In [74]:
Numpub_journal_first_author_distribution .columns

Index(['Journal', 'work_id', 'author_id'], dtype='object')

In [82]:
Numpub_journal_first_author_distribution_Nature = Numpub_journal_first_author_distribution[Numpub_journal_first_author_distribution['Journal']=='Nature']
Numpub_journal_first_author_distribution_Nature['Totalfirstauthor'] = 91387
Numpub_journal_first_author_distribution_Nature['AuthorShare'] = Numpub_journal_first_author_distribution_Nature['author_id']/Numpub_journal_first_author_distribution_Nature['Totalfirstauthor'] * 100

Numpub_journal_first_author_distribution_Science = Numpub_journal_first_author_distribution[Numpub_journal_first_author_distribution['Journal']=='Science']
Numpub_journal_first_author_distribution_Science['Totalfirstauthor'] = 73190
Numpub_journal_first_author_distribution_Science['AuthorShare'] = Numpub_journal_first_author_distribution_Science['author_id']/Numpub_journal_first_author_distribution_Science['Totalfirstauthor'] * 100

Numpub_journal_first_author_distribution_PNAS = Numpub_journal_first_author_distribution[Numpub_journal_first_author_distribution['Journal']=='Proceedings of the National Academy of Sciences']
Numpub_journal_first_author_distribution_PNAS['Totalfirstauthor'] = 99099
Numpub_journal_first_author_distribution_PNAS['AuthorShare'] = Numpub_journal_first_author_distribution_PNAS['author_id']/Numpub_journal_first_author_distribution_PNAS['Totalfirstauthor'] * 100


/var/folders/4g/t694bx_s0353zx81yggk1p680000gn/T/ipykernel_60722/1002943365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Numpub_journal_first_author_distribution_Nature['Totalfirstauthor'] = 91387
/var/folders/4g/t694bx_s0353zx81yggk1p680000gn/T/ipykernel_60722/1002943365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Numpub_journal_first_author_distribution_Nature['AuthorShare'] = Numpub_journal_first_author_distribution_Nature['author_id']/Numpub_journal_first_author_distribution_Nature['Total

In [83]:
Numpub_journal_first_author_distribution_PNAS .head(15)

,Journal,work_id,author_id,Totalfirstauthor,AuthorShare
65,Proceedings of the National Academy of Sciences,1,83988,99099,84.751612
66,Proceedings of the National Academy of Sciences,2,11117,99099,11.218075
67,Proceedings of the National Academy of Sciences,3,2458,99099,2.480348
68,Proceedings of the National Academy of Sciences,4,748,99099,0.754801
69,Proceedings of the National Academy of Sciences,5,306,99099,0.308782
70,Proceedings of the National Academy of Sciences,6,148,99099,0.149346
71,Proceedings of the National Academy of Sciences,7,81,99099,0.081736
72,Proceedings of the National Academy of Sciences,8,61,99099,0.061555
73,Proceedings of the National Academy of Sciences,9,36,99099,0.036327
74,Proceedings of the National Academy of Sciences,10,27,99099,0.027245
